In [1]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
off_test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
off_train = off_train.dropna(subset = ['Coupon_id'])

In [2]:
off_train['result'] = off_train[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

# 用户有当天使用优惠券,Use_today

In [3]:
date_nan_drop = off_train.dropna(subset=['Date'])
date_nan_drop = date_nan_drop.reset_index()

In [4]:
import datetime

def Use(z):
    if z.Date_received == z.Date:
        return 1
    else:
        return np.NaN

date_nan_drop['Use_today'] = date_nan_drop.apply(Use, axis=1)

In [5]:
a = date_nan_drop[['User_id', 'Use_today']]
a = a.drop_duplicates(subset=['User_id'])

In [6]:
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')

# 只在一个商铺领取过优惠劵并且有消费记录,Use_One_Merchant

In [7]:
Consume_oneMerchant = date_nan_drop['Merchant_id'].groupby(date_nan_drop['User_id'])

In [8]:
a = Consume_oneMerchant.count()
a = a.reset_index()
a = a.rename(columns={'Merchant_id':'Use_One_Merchant'})
a['Use_One_Merchant'] = a.Use_One_Merchant.apply(lambda x:1 if x == 1 else 0)

In [9]:
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')

In [ ]:
user_coupon = off_train[['User_id', 'Coupon_id']]
# 利用User_id和Coupon_id两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge Coupon_size代表用户领取的优惠劵数目
off_train = off_train.merge(user_coupon_count, on='User_id', how='inner')

user_coupon = off_train[['User_id', 'result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目（这里相当于求和），并进行序号重置
user_coupon_used_count = user_coupon.groupby(by=['User_id']).sum().reset_index()
# 重命名 Consume_count用户使用了的优惠劵数目
user_coupon_used_count.rename(columns={'result':'Conupon_have_used_count'}, inplace=True)
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_used_count, on='User_id', how='inner')

# 为训练集添加Coupon_all_consume特征，标记那些领取优惠劵数目等于使用优惠劵数目
off_train['Coupon_all_consume'] = off_train[['Conupon_have_used_count','Coupon_size']].apply(lambda x : 1 if x[0] == x[1] else 0, axis = 1)
# 去掉产生的中间列
off_train = off_train.drop(columns=['Coupon_size', 'Conupon_have_used_count'])
# 提取需要的列好merge到测试集中
off_train_coupon_all_consume = off_train[['User_id', 'Coupon_all_consume']]
off_train_coupon_all_consume = off_train_coupon_all_consume.drop_duplicates(['User_id', 'Coupon_all_consume'])
#为测试集添加Coupon_all_consume特征
off_test = off_test.merge(off_train_coupon_all_consume, on='User_id', how='left')
# 填充merge以后的空值为0
off_test = off_test.fillna({'Coupon_all_consume':0})


In [ ]:
user_coupon = off_train[['User_id', 'Coupon_id','result']]
# 利用User_id和result两列对用户进性分组，并且计算每组的数目，并进行序号重置
user_coupon_count = user_coupon.groupby(by=['User_id','result']).size().reset_index()
# 重命名
user_coupon_count.rename(columns={0:'Coupon_size'}, inplace=True)
# 筛选出领取次数为1并且未消费的
user_coupon_count_1 = user_coupon_count[(user_coupon_count.Coupon_size == 1) & (user_coupon_count.result == 0)]
# 去掉不需要merge的这一列，即result
user_coupon_count_1 = user_coupon_count_1.drop(columns=['result'])

# 训练集进行添加One_coupon_no_consume特征
# 将筛选出来需要merge的列和原始的数据进行merge
off_train = off_train.merge(user_coupon_count_1, on='User_id', how='left')
#再次重命名
off_train.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
# 填充merge以后的空值项为 0
off_train = off_train.fillna({'One_coupon_no_consume':0})

# 测试集添加One_coupon_no_consume特征
off_test = off_test.merge(user_coupon_count_1, on='User_id', how='left')
off_test.rename(columns={'Coupon_size':'One_coupon_no_consume'}, inplace=True)
off_test = off_test.fillna({'One_coupon_no_consume':0})


In [10]:
#按用户对优惠券id和消费日期进行分组并计数，得到用户的优惠券消费次数
User_Coupon_notna = off_train['Date'].groupby(off_train['User_id']).count()
User_Coupon_notna = User_Coupon_notna.reset_index()
User_Coupon_notna.rename(columns = {'Date' : 'Consume_count'}, inplace=True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Coupon_notna[['User_id', 'Consume_count']], on = ['User_id'], how = 'left')
off_test = pd.merge(off_test, User_Coupon_notna[['User_id', 'Consume_count']], on = 'User_id', how = 'left')
off_test = off_test.fillna(value = {'Consume_count' : 0})

In [11]:
#对训练集和测试集中的Discount_rate进行独热编码
off_train = pd.get_dummies(off_train, prefix=['Discount_rate'])
off_test = pd.get_dummies(off_test, prefix=['Discount_rate'])

#训练集中出现了4个测试集中没有出现的码段，进行填充
m = list(set(off_train.columns.tolist()) - set(off_test.columns.tolist()))
for i in m:
    off_test[i] = 0

#测试集中有一个训练集中没有的码段，进行填充    
off_train['Discount_rate_500:30'] = 0

In [12]:
off_train.Distance = off_train.Distance.fillna(1)
off_test.Distance = off_test.Distance.fillna(1)

In [13]:
off_test = off_test.fillna(0)
off_train = off_train.fillna(0)

In [14]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression

#调用逻辑回归模型，并进行训练
clf = LogisticRegression()
clf.fit(x_train, y_train)

#进行预测，并将需要保存的部分写入save
predict = clf.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

#输出结果为csv
save.to_csv('E:/tianchi/20180725/Use_today_one_merchant.csv', index = None)

# 计算5折交叉准确率

In [15]:
from sklearn.model_selection import cross_val_score

clf = LogisticRegression()
cross_val_score(clf, x_train, y_train, cv=5)

array([0.93537362, 0.93577237, 0.93502203, 0.93488436, 0.93451409])

# 抽取1/5数据做验证集，计算auc

In [16]:
from sklearn import cross_validation,metrics

train_x, test_x, train_y, test_y = cross_validation.train_test_split(x_train, y_train, test_size=0.2)#把训练集按0.2的比例划分为训练集和验证集

clf = LogisticRegression()
clf.fit(train_x, train_y)
predict = clf.predict_proba(test_x)
test_auc = metrics.roc_auc_score(test_y, predict[:,1])
test_auc 

C:\Users\pulei\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.9535294981847389

# 画学习曲线

In [17]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

